In [6]:
import numpy as np
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import linear_model# import LogisticRegression
from subprocess import check_output

Load and merge datasets, meet data

In [7]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
data = train.append(test, sort=False)
data.sample(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
323,324,1.0,2,"Caldwell, Mrs. Albert Francis (Sylvia Mae Harb...",female,22.0,1,1,248738,29.00,NaN,S
387,388,1.0,2,"Buss, Miss. Kate",female,36.0,0,0,27849,13.00,NaN,S
267,1159,NaN,3,"Warren, Mr. Charles William",male,NaN,0,0,C.A. 49867,7.55,NaN,S


See types and incomplete features

In [17]:
pd.DataFrame([data.dtypes, data.count()])

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,int64,float64,int64,object,object,float64,int64,int64,object,float64,object,object
1,1309,891,1309,1309,1309,1046,1309,1309,1309,1308,295,1307


In [5]:
def fillna_mean(feature: pd.Series):
    mean = feature.mean()
    feature.fillna(mean, inplace=True)
    
fillna_mean(data['Fare'])
fillna_mean(data['Age'])

In [6]:
# new features - FamilySize
data['FamilySize'] = data['SibSp'] + data['Parch'] + 1

In [7]:
# new features - Alone
data['Alone'] = 0
data.loc[data['FamilySize'] == 1, 'Alone'] = 1

In [8]:
# extract titles from passenger names
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return ''

# new feature - Title
data['Title'] = data['Name'].apply(get_title)
data['Title'].replace('Mlle', 'Miss', inplace=True)
data['Title'].replace('Ms', 'Miss', inplace=True)
data['Title'].replace('Mme', 'Mrs', inplace=True)
data['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare', inplace=True)

In [9]:
def encode_labels(label: str):
    le = LabelEncoder()
    data[label].fillna('', inplace=True)
    le.fit(data[label])
    data.loc[:, label] = le.transform(data[label])

encode_labels('Sex')
encode_labels('Embarked')
encode_labels('Title')

In [10]:
train = data[data['Survived'].notna()]
test = data[data['Survived'].isna()]

In [11]:
# prepare train for fitting
# other features appears useless
features = ['Age', 'Sex', 'Pclass', 'Fare', 'Parch', 'Alone']
X = train.loc[:, features]
Y = train.loc[:, 'Survived']
XTrain, XValid, YTrain, YValid = train_test_split(X, Y, test_size=0.2, random_state=40)

In [12]:
# Fit logistic regression using scikit
LR = linear_model.LogisticRegression(C=1000, solver='lbfgs', max_iter=1000)
LR.fit(X=XTrain, y=YTrain)

def accuracy(Y: np.array, yPred: np.array) -> float:
  return np.sum(yPred==Y) / len(Y)

# Use model to predict on training and validation sets
print('     Train accuracy', accuracy(YTrain, LR.predict(XTrain)))
print('Validation accuracy', accuracy(YValid, LR.predict(XValid)))

     Train accuracy 0.7963483146067416
Validation accuracy 0.8268156424581006


In [13]:
# Predict for test set
# Create a Kaggle submission
XTest = test.loc[:, features]
YTest = LR.predict(XTest)
#data['Survivd'] = data['Survived'].astype('int64')
sub = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': YTest.astype('int64')})
sub.to_csv('submission.csv', index=False)